In [79]:
#예측 모델
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
import lightgbm as lgb
import eli5
from eli5.sklearn import PermutationImportance
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
#기초 모듈
import numpy as np
import pandas as pd


#평가지표
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
#그림
import seaborn as sns
import matplotlib.pyplot as plt
#ignore warning
import warnings
warnings.filterwarnings("ignore")
#GridsearchCV


In [49]:
#추가 정보는 유역, 저수, 침수
Addinfo={"BS": [1309.7, 457, 10.75], "HM": [763, 466, 320], "SU": [3383, 466, 6], "PN": [374.3, 466, 4], "YC": [565, 466, 0], "GC": [1010, 457, 3.65]}

motherdir2="C:/Users/infec/Desktop/수자원공사 공모전/사진파일/"
#수위, 유역,  저수용량, 침수
RegCoef=[-7.08457020e+05, 6.46888454e+01, 5.62264433e+03, 2.08078043e+04]

In [85]:
totaldf=pd.read_csv("C:/Users/infec/Desktop/수자원공사 공모전/사진파일/Totaldf.csv", encoding="cp949")
JA=totaldf[["Date", "주암댐"]]
JA.set_index(["Date"], inplace=True)


,주암댐
Date,
2019-06-21,13.74
2019-06-22,12.84
2019-06-23,13.84
2019-06-24,13.59
2019-06-25,12.57
...,...
2021-09-04,18.35
2021-09-05,17.84
2021-09-06,17.88


In [3]:
Totaldf=pd.read_csv(motherdir2+"Totaldf.csv")
df2021=pd.read_csv(motherdir2+"DF2021.csv")

In [4]:
GC_level_est=RandomForestRegressor(bootstrap=True, max_depth=3, min_samples_split=3, n_estimators=30 )
BS_rf=RandomForestRegressor(bootstrap=True, max_depth=3, min_samples_split=3, n_estimators=30)

BS_poly=Pipeline([("poly", PolynomialFeatures(degree=1)), ("linear", LinearRegression())])

BS_level_est=VotingRegressor([("RF", BS_rf), ("POLY", BS_poly)])
SU_level_est=RandomForestRegressor(bootstrap=True, max_depth=7, min_samples_split=3, n_estimators=40)
YC_level_est=RandomForestRegressor(bootstrap=True, max_depth=3, min_samples_split=3, n_estimators=40)
HM_level_est=RandomForestRegressor(bootstrap=True, min_samples_split=3, n_estimators=50, max_depth=3)
PN_level_est=RandomForestRegressor(bootstrap=True, max_depth=3, min_samples_split=3, n_estimators=80)


In [5]:
#outlier 제거하는 함수 만들기 
def remove_outlier(point, weight=1.5, year=None):
    data=year[[point]]
    level_1q=data.quantile(0.25)
    level_3q=data.quantile(0.75)
    IQR=level_3q-level_1q
    data=data[(data<=level_3q+(weight*IQR) )&( data>=level_1q-(weight*IQR))]
    data.dropna(inplace=True)
    return data
def find_outlier(point, weight=1.5, year=None):
    data=year[[point]]
    level_1q=data.quantile(0.25)
    level_3q=data.quantile(0.75)
    IQR=level_3q-level_1q
    data=data[(data>level_3q+(weight*IQR) ) |( data<level_1q-(weight*IQR))]
    data.dropna(inplace=True)
    return data
#광천교를 예시로 한번 해보자 

#학습 위한 함수 작성
def tillsplit(point=None, spec=None, spw=1.5, dam=None):
    y_data=remove_outlier(point, weight=spw, year=spec)
    x=Totaldf[[dam]].loc[y_data.index][[dam]]
    y=y_data.copy()
    y=y.values
    X_train, X_test, y_train, y_test=train_test_split(x, y, test_size=0.2, random_state=156)
    return X_train, X_test, y_train, y_test, x, y
def twosplit(point=None, spec=None, spw=1.5, dam=None):
    y_data=remove_outlier(point, weight=spw, year=spec)
    x=Totaldf[dam].loc[y_data.index].values
    y=y_data.copy()
    y=y.values
    X_train, X_test, y_train, y_test=train_test_split(x, y, test_size=0.2, random_state=156)
    return X_train, X_test, y_train, y_test, x, y

#평가지표 별 점수 나타낼 함수 만들기
def RMSE(value):
    return np.sqrt(value)
scorename=["RMSE", "R2", "MSLE"]
def accuracy_score(test=None, real=None):
    global scorelist
    scorelist=[]
    rmse=RMSE(mean_squared_error(test, real))
    print("RMSE score: ", rmse, "\n")
    r2=r2_score(test, real)
    print("r2 score: ", r2, "\n")
    msle=mean_squared_log_error(test, real)
    print("MSLE: ", msle)
    scorelist.append(rmse)
    scorelist.append(r2)
    scorelist.append(msle)
    return scorelist

In [6]:
X_train, X_test, y_train, y_test, total_x, total_y=tillsplit(point="PN", spec=df2021, spw=1.5, dam="섬진강")
PN_level_est.fit(X_train, y_train)
PNpred=PN_level_est.predict(X_test)


In [7]:
HM_level_est.fit(X_train, y_train)
HMpred=HM_level_est.predict(X_test)


In [8]:
X_train, X_test, y_train, y_test, total_x, total_y=twosplit(point="YC", spec=df2021, spw=1.5, dam=["섬진강","주암댐"])
YC_level_est.fit(X_train, y_train)
YCpred=YC_level_est.predict(X_test)


In [9]:
BS_level_est.fit(X_train, y_train)
BSpred=BS_level_est.predict(X_test)


In [10]:
GC_level_est.fit(X_train, y_train)
GCpred=GC_level_est.predict(X_test)


In [11]:
SU_level_est.fit(X_train, y_train)
SUpred=SU_level_est.predict(X_test)


In [36]:
easy={"SU": SU_level_est, "GC": GC_level_est, "YC": YC_level_est, "HM": HM_level_est, "PN": PN_level_est, "BS": BS_level_est}
def ModelA(point="", release=[0., 0. ]):
    
    if point in ["HM", "PN"]:
        model=easy[point]
        X_data=np.array(release[0]).reshape(-1, 1)
        level_est=model.predict(X_data)
    else:
        model=easy[point]
        X_data=np.array(release).reshape(1, -1)
        level_est=model.predict(X_data)
    return level_est


In [37]:
#example
ModelA(point="YC", release=[31.67, 32.2])

array([2.19645249])

In [43]:
def ModelB(point="", level_est=0):
    infolist=Addinfo[point]
    return RegCoef[0]*level_est+RegCoef[1]* infolist[0]+RegCoef[2]* infolist[1]+RegCoef[3]* infolist[2]+209621.75162723

In [80]:
#연습
PR=ModelA(point="HM", release=[31.67, 32.2])
ModelB(point="HM", level_est=PR)

array([8296301.09584141])

In [89]:
Best_time_fit=SimpleExpSmoothing(JA, initialization_method="heuristic").fit(smoothing_level=0.995, optimized=False)
def ModelC():
    origin=Best_time_fit.fittedvalues
    fcast=Best_time_fit.forecast(13)
    file= pd.concat([origin, fcast], axis=0)
    exp=file[-1:]
    return file, exp

In [93]:
file1, exp=ModelC()
file1.tail(20)

2021-09-02    19.231261
2021-09-03    19.080756
2021-09-04    18.662104
2021-09-05    18.351561
2021-09-06    17.842558
2021-09-07    17.879813
2021-09-08    17.780499
160           17.352152
161           17.352152
162           17.352152
163           17.352152
164           17.352152
165           17.352152
166           17.352152
167           17.352152
168           17.352152
169           17.352152
170           17.352152
171           17.352152
172           17.352152
dtype: float64

In [100]:
def ModelD(point="", release=[0.0, 0.0]):
    level_est=ModelA(point=point, release=release)
    damage_est=ModelB(point=point, level_est=level_est)
    return damage_est


#case 1 : 11일전 방류량이 [31.67, 17.7804]일 때

file1, exp=ModelC()

ModelD(point="HM", release=[31.67, exp])

array([8296301.09584141])

array([16674039.74147761])

In [103]:
#모든 지점에 대한 ModelC값 표현하기 
#기준은 m^3/s
def everymodelD(release=[0.0, 0.0]):
    for i in Addinfo.keys():
        print( i, "지점의 11일 후 피해액은 ", (ModelD(point=i, release=release)).reshape(-1), "입니다")
        

In [102]:
print('오늘 섬진강댐 방류량은 31.67, 주암댐 방류량은 17.7804 입니다')
everymodelD(release=[31.67, exp])

오늘 섬진강댐 방류량은 31.67, 주암댐 방류량은 17.7804 입니다
BS 지점의 11일 후 피해액은  [1522324.5477196] 입니다
HM 지점의 11일 후 피해액은  [8296301.09584141] 입니다
SU 지점의 11일 후 피해액은  [1629116.2318148] 입니다
PN 지점의 11일 후 피해액은  [1688509.1589952] 입니다
YC 지점의 11일 후 피해액은  [1314110.47406246] 입니다
GC 지점의 11일 후 피해액은  [1364465.69782509] 입니다


RandomForestRegressor(max_depth=7, min_samples_split=3, n_estimators=40)